In [10]:
DATA='/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2020-02-14'

In [11]:
for (fold in c(0)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.csv',sep=''),
              sep=',',header=TRUE)
    df_train=df[which(df[,foldname]=="True"),]
    print(dim(df_train))
    
    set.seed(1234)
    
    model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
    A549_p53WT_Nutlin.3_Z.score+
    A549_p53NULL_Nutlin.3_Z.score+
    A549_p53NULL_Etoposide_Z.score'
    model_def=paste(model_def,'+0',sep='')
    myglm <- glm(data=df_train,model_def, family = poisson())

    #make predictions for KRAS
    df_kras=read.table('/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2019-06-07/KRAS/mutational_signatures/KRAS_mutationalSignatures_Andrew.txt',
    sep='\t',header=TRUE)
    rownames(df_kras)=df_kras[,'Mutation']
    df_kras[,'A549_p53WT_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Etoposide_Z.score']=0
    
    
    df_kras['predicted.mutonly.response']=predict(myglm,df_kras,type='response')
    pred_kras=rowSums(predict(myglm,df_kras,type='terms'))
    
    df_kras['predicted.mutonly.terms']=pred_kras
    write.table(df_kras,file=paste(DATA,'/mutfreq/KRAS.predictedFromTP53.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)
    
    
}


[1] 0
[1] 2046   47


In [12]:
#KRAS models

for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='COSMIC.1 ~ mutational_signature_score+
        GILA_day7'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='COSMIC.1 ~ GILA_day7'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='COSMIC.1 ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='COSMIC.1 ~ mutational_signature_score+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, paste(model_def,'+0',sep=''), 
                     family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}

[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 24 58
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 84 59
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 84 60
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 24 61
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 84 62
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 84 63
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1] 24 64
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1] 24 65
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 29 62
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 85 63
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 85 64
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 29 65
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 85 66
[1] "---

In [20]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
        A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='IARC_Mutation_Counts ~ A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        model_def=paste(model_def,'+0',sep='')
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}


[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 31 85
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 1208   86
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 1208   87
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 31 88
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 1208   89
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 1208   90
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1] 31 91
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1] 31 92
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 34 85
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 1207   86
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 1207   87
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 34 88
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) tes

In [11]:
for (fold in c(0)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.csv',sep=''),
              sep=',',header=TRUE)
    df_train=df[which(df[,foldname]=="True"),]
    ########
    df_train=df
    print(dim(df_train))
    
    set.seed(1234)
    
    model_def='IARC_Mutation_Counts ~ Signature_1+
    Signature_2+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_13+
    Signature_24+
    A549_p53WT_Nutlin.3_Z.score+
    A549_p53NULL_Nutlin.3_Z.score+
    A549_p53NULL_Etoposide_Z.score'
    model_def=paste(model_def,'+0',sep='')
    myglm <- glm(data=df_train,model_def, family = poisson())
    
    #prediction full model
    df[,'A549_p53WT_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df[,'A549_p53NULL_Etoposide_Z.score']=0
    pred=predict(myglm,df,type='response')
    df['predicted.mutonly.response']=pred
    
    #prediction only based on mutational signatures
    df_mutonly=df
    df_mutonly[,c('A549_p53WT_Nutlin.3_Z.score',
    'A549_p53NULL_Nutlin.3_Z.score',
    'A549_p53NULL_Etoposide_Z.score')]=0
    pred_mutonly=rowSums(predict(myglm,df_mutonly,type='terms'))#predict(myglm,df_mutonly,type='link')
    df['predicted.mutonly.terms']=pred_mutonly
    
    #write predictions 
    preds=df[,c('Allele','predicted.mutonly.response','predicted.mutonly.terms')]
    write.table(preds,file=paste(DATA,'/mutfreq/TP53.training_data.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)
    
    #make predictions for KRAS
    df_kras=read.table('/ahg/regevdata/projects/bn10_cancer_variants/bn10_oana_2018-06-22/results/2018-06-22/results/2019-06-07/KRAS/mutational_signatures/KRAS_mutationalSignatures_Andrew.txt',
    sep='\t',header=TRUE)
    rownames(df_kras)=df_kras[,'Mutation']
    df_kras[,'A549_p53WT_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Nutlin.3_Z.score']=0
    df_kras[,'A549_p53NULL_Etoposide_Z.score']=0
    
    
    df_kras['predicted.mutonly.response']=predict(myglm,df_kras,type='response')
    pred_kras=rowSums(predict(myglm,df_kras,type='terms',terms=c('Signature_1',
                                           'Signature_2',
                                           'Signature_4',
                                           'Signature_5',
                                           'Signature_6',
                                           'Signature_7',
                                           'Signature_13',
                                           'Signature_24')))
    
    df_kras['predicted.mutonly.terms']=pred_kras
    write.table(df_kras,file=paste(DATA,'/mutfreq/KRAS.predictedFromTP53.',foldname,'.pred.csv',sep=''),
               quote=FALSE,row.names=FALSE,col.names=TRUE)
    
}



[1] 0
[1] 2414   47


In [9]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='IARC_Mutation_Counts ~ mutational_signature_score+
        A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='IARC_Mutation_Counts ~ A549_p53WT_Nutlin.3_Z.score+
        A549_p53NULL_Nutlin.3_Z.score+
        A549_p53NULL_Etoposide_Z.score'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='IARC_Mutation_Counts ~ mutational_signature_score+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/TP53.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}


[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 40 50
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 700  51
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 700  52
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 40 53
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 700  54
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 700  55
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1] 40 56
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1] 40 57
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 37 54
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 697  55
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 697  56
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 37 57
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 69

In [12]:
#KRAS models

for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='COSMIC.1 ~ mutational_signature_score+
        GILA_day7'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='COSMIC.1 ~ GILA_day7'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='COSMIC.1 ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='COSMIC.1 ~ mutational_signature_score+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, paste(model_def,'+0',sep=''), 
                     family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}

[1] 0
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 24 94
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 84 95
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 84 96
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 24 97
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 84 98
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(L)"
[1] 84 99
[1] "------------------"
[1] "Cell- Mut- sc-eVIP+"
[1]  24 100
[1] "------------------"
[1] "Cell- Mut+ sc-eVIP+"
[1]  24 101
[1] 1
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(S) test(S)"
[1] 29 94
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(S)"
[1] 85 95
[1] "------------------"
[1] "Cell+ Mut+ sc-eVIP- train(L) test(L)"
[1] 85 96
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(S) test(S)"
[1] 29 97
[1] "------------------"
[1] "Cell+ Mut- sc-eVIP- train(L) test(S)"
[1] 85 98
[1] 

In [ ]:
for (fold in c(0:9)){
    print(fold)
    foldname=paste('fold',as.character(fold),sep='')
    df=read.table(paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.csv',sep=''),
                  sep=',',header=TRUE)

    for (modelname in c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut+ sc-eVIP-',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        #'Cell+ Mut- sc-eVIP-',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
        print('------------------')
        print(modelname)
        
        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-')){
            
            model_def='COSMIC.1 ~ Signature_1+
    Signature_2+
    Signature_3+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_8+
    Signature_9+
    Signature_10+
    Signature_11+
    Signature_12+
    Signature_13+
    Signature_14+
    Signature_15+
    Signature_16+
    Signature_17+
    Signature_18+
    Signature_19+
    Signature_20+
    Signature_21+
    Signature_22+
    Signature_23+
    Signature_24+
    Signature_25+
    Signature_26+
    Signature_27+
    Signature_28+
    Signature_29+
    Signature_30+
        GILA_day7'
        }
        
        if (modelname %in% c('Cell+ Mut- sc-eVIP- train(S) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            model_def='COSMIC.1 ~ GILA_day7'   
        }
    
        if (modelname %in% c('Cell- Mut- sc-eVIP+')){
            model_def='COSMIC.1 ~ HotellingT2'
        }
    
        if (modelname %in% c('Cell- Mut+ sc-eVIP+')){
            model_def='COSMIC.1 ~ Signature_1+
    Signature_2+
    Signature_3+
    Signature_4+
    Signature_5+
    Signature_6+
    Signature_7+
    Signature_8+
    Signature_9+
    Signature_10+
    Signature_11+
    Signature_12+
    Signature_13+
    Signature_14+
    Signature_15+
    Signature_16+
    Signature_17+
    Signature_18+
    Signature_19+
    Signature_20+
    Signature_21+
    Signature_22+
    Signature_23+
    Signature_24+
    Signature_25+
    Signature_26+
    Signature_27+
    Signature_28+
    Signature_29+
    Signature_30+
        HotellingT2'
        }


        if (modelname %in% c('Cell+ Mut+ sc-eVIP- train(S) test(S)',
                        
                        'Cell+ Mut- sc-eVIP- train(S) test(S)',
                        
                        'Cell- Mut- sc-eVIP+',
                        'Cell- Mut+ sc-eVIP+')){
            df_train=df[which(df[,paste('train',foldname,sep='')]=="True"),]
        }

        if (modelname %in% c(
                        'Cell+ Mut+ sc-eVIP- train(L) test(S)',
                        'Cell+ Mut+ sc-eVIP- train(L) test(L)',
                        'Cell+ Mut+ sc-eVIP-',
                        
                        
                        'Cell+ Mut- sc-eVIP- train(L) test(S)',
                        'Cell+ Mut- sc-eVIP- train(L) test(L)',
                        'Cell+ Mut- sc-eVIP-')){
            df_train=df[which(df[,foldname]=="True"),]
        }
        
        print(dim(df_train))

        set.seed(1234)
        myglm <- glm(data=df_train, model_def, family = poisson())

        #prediction full model
        pred=predict(myglm,df,type='response')
        #df['predicted']=pred

        #write predictions 
        df[modelname]=pred
        

    }
    write.table(df,file=paste(DATA,'/mutfreq/KRAS.model_comparison.',foldname,'.pred.csv',sep=''),
                   quote=FALSE,row.names=FALSE,col.names=TRUE,sep='\t')

}